<a href="https://colab.research.google.com/github/YossiAsher/abstract-learning-in-image-processing/blob/main/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install drawSvg
import drawSvg as draw

In [ ]:
import cv2

import numpy as np
import glob

import os

from pathlib import Path


In [ ]:
def first_nonzero(arr, axis, invalid_val=-1):
    arr = np.sum(arr, 0)
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

In [ ]:
def crop_image(img):
  height, width = img.shape

  a1 = first_nonzero(img, 0)

  img_90 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
  a2 = first_nonzero(img_90, 0)

  img_180 = cv2.rotate(img_90, cv2.ROTATE_90_COUNTERCLOCKWISE)
  a3 = first_nonzero(img_180, 0)

  img_270= cv2.rotate(img_180, cv2.ROTATE_90_COUNTERCLOCKWISE)
  a4 = first_nonzero(img_270, 0)

  return img[a2:height - a4, a1:width- a3]

In [ ]:
def save_svg(img, name):
  height, width = img.shape
  d = draw.Drawing(width, height)
  contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_TC89_L1)
  for c in contours:
    p = draw.Path()
    for i in range(len(c)):
      x, y = c[i][0]
      if i == 0:
        p.M(x, y)
      else:
        p.L(x, y)
    d.append(p)
  d.saveSvg(name)

In [ ]:
def save_png(img, name):
  cv2.imwrite(name, img)

In [ ]:
davis_dataset_path = 'png' # need to replace
base_path = 'processing' 
files = glob.glob(f"{davis_dataset_path}/**/*.png", recursive=True)

In [ ]:
for file in files:
  clas = file.split('/')[-2]
  file_name = file.split('/')[-1].split('.')[0]
  img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
  ret, mask = cv2.threshold(img,27,25, cv2.THRESH_BINARY)
  mask = crop_image(mask)

  path_svg = f'{base_path}/svg/{clas}'
  Path(path_svg).mkdir(parents=True, exist_ok=True)
  save_svg(mask, f"{path_svg}/{file_name}.svg")

  path_png = f'{base_path}/png/{clas}'
  Path(path_png).mkdir(parents=True, exist_ok=True)
  save_png(mask, f"{path_png}/{file_name}.png")
  break